In [1]:
from sqlalchemy import create_engine
from config import Config
from pandas import read_sql
from transformers import BinaryEncoder, CityExtractor, ColDropper, XYTrainTestSplitter, PredictionTransformer, Scorer

eng = create_engine(Config.DATABASE_URI)
car_heat = read_sql(sql='select * from car_heat', con=eng)

#drop columns which currently are not processed
dropper = ColDropper()
dropper.transform(data=car_heat, columns=['date', 'match_hash', 'heat_id'])

#extract the time value so that it's a string and not python datetime object
car_heat['time'] = [x.strftime("%H:%M") for x in car_heat['time']]

#remove club names and change them to only cities to account for clubs changing sponsors and names as a result
c_ex = CityExtractor()
c_ex.transform(car_heat)

#create binary variables for categorical columns
encoder = BinaryEncoder()
cat_cols = ['a_rider', 'b_rider', 'c_rider', 'd_rider', 'stadium', 'round', 'year', 'time', 'name_team_home', 'name_team_away']
encoder.fit_transform(data=car_heat, columns=cat_cols)

#split into X, Y, test and train
splitter = XYTrainTestSplitter()
X_train, Y_train, X_test, Y_test = splitter.transform(data=car_heat, y_columns = ['a_score','b_score','c_score','d_score'])

C:\Users\kucza\.virtualenvs\analytics-KSLE3GME\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [2]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train, Y_train)

RandomForestRegressor()

In [3]:
pred = rfr.predict(X_test)

In [4]:
pred_trans = PredictionTransformer()
predictions = pred_trans.transform(pred)

In [5]:
predictions

array([[2, 1, 0, 3],
       [1, 2, 0, 3],
       [3, 2, 0, 1],
       ...,
       [1, 3, 2, 0],
       [1, 2, 0, 3],
       [3, 0, 2, 1]])

In [6]:
pred

array([[2.2 , 0.89, 0.5 , 2.41],
       [1.42, 1.49, 1.25, 1.83],
       [2.41, 2.09, 0.61, 0.89],
       ...,
       [1.63, 1.78, 1.65, 0.92],
       [0.82, 1.95, 0.54, 2.61],
       [1.9 , 0.48, 1.85, 1.76]])